In [ ]:
pip install scikit-image

In [6]:
pip install nibabel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 23.2 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
import numpy as np
import skimage.transform as skTrans
import nibabel as nib
import pandas as pd
import os
import sys
import time


In [10]:
import os
import shutil

# 定义嵌套文件目录和目标目录
source_dir = "ADNI/"  # 当前嵌套结构的根目录
target_dir = "flattened_ADNI2/"  # 扁平化后的目标目录

# 确保目标目录存在
os.makedirs(target_dir, exist_ok=True)

# 遍历所有子目录，提取 .nii 文件
for root, dirs, files in os.walk(source_dir):
    for file in files:
        if file.endswith(".nii"):
            source_path = os.path.join(root, file)
            target_path = os.path.join(target_dir, file)
            shutil.copy(source_path, target_path)
            print(f"Copied: {source_path} -> {target_path}")

print("Flattening complete!")


Copied: ADNI/023_S_4501/MT1__GradWarp__N3m/2013-05-09_11_31_51.0/I373120/ADNI_023_S_4501_MR_MT1__GradWarp__N3m_Br_20130520142506859_S189065_I373120.nii -> flattened_ADNI2/ADNI_023_S_4501_MR_MT1__GradWarp__N3m_Br_20130520142506859_S189065_I373120.nii
Copied: ADNI/127_S_4148/MT1__GradWarp__N3m/2012-08-30_09_08_04.0/I338591/ADNI_127_S_4148_MR_MT1__GradWarp__N3m_Br_20121008090545855_S166004_I338591.nii -> flattened_ADNI2/ADNI_127_S_4148_MR_MT1__GradWarp__N3m_Br_20121008090545855_S166004_I338591.nii
Copied: ADNI/029_S_4290/MT1__GradWarp__N3m/2012-12-06_12_06_09.0/I358268/ADNI_029_S_4290_MR_MT1__GradWarp__N3m_Br_20130206161210233_S180242_I358268.nii -> flattened_ADNI2/ADNI_029_S_4290_MR_MT1__GradWarp__N3m_Br_20130206161210233_S180242_I358268.nii
Copied: ADNI/127_S_5056/MT1__GradWarp__N3m/2013-01-03_13_40_50.0/I353890/ADNI_127_S_5056_MR_MT1__GradWarp__N3m_Br_20130108094106288_S178820_I353890.nii -> flattened_ADNI2/ADNI_127_S_5056_MR_MT1__GradWarp__N3m_Br_20130108094106288_S178820_I353890.nii


In [11]:
def normalize_img(img_array):
    maxes = np.quantile(img_array,0.995,axis=(0,1,2))
    #print("Max value for each modality", maxes)
    return img_array/maxes

def create_dataset(meta, meta_all,path_to_datadir):
    files = os.listdir(path_to_datadir)
    start = '_'
    end = '.nii'
    for file in files:
        print(file)
        if file != '.DS_Store':
            path = os.path.join(path_to_datadir, file)
            print(path)
            img_id = file.split(start)[-1].split(end)[0]
            idx = meta[meta["Image Data ID"] == img_id].index[0]
            im = nib.load(path).get_fdata()
            n_i, n_j, n_k = im.shape
            center_i = (n_i - 1) // 2  
            center_j = (n_j - 1) // 2
            center_k = (n_k - 1) // 2
            im1 = skTrans.resize(im[center_i, :, :], (72, 72), order=1, preserve_range=True)
            im2 = skTrans.resize(im[:, center_j, :], (72, 72), order=1, preserve_range=True)
            im3 = skTrans.resize(im[:, :, center_k], (72, 72), order=1, preserve_range=True)
            im = np.array([im1,im2,im3]).T
            label = meta.at[idx, "Group"]
            subject = meta.at[idx, "Subject"]
            norm_im = normalize_img(im)
            meta_all = pd.concat([meta_all, pd.DataFrame([{"img_array": im, "label": label, "subject": subject}])], ignore_index=True)
    meta_all.to_pickle("mri_meta_v.pkl")

            

In [12]:

path_to_datadir = 'flattened_ADNI2/'
meta = pd.read_csv('MRI_meta_v.csv')
print("opened meta")
print(len(meta))
#get rid of not needed columns
meta = meta[["Image Data ID", "Group", "Subject"]] #MCI = 0, CN =1, AD = 2
meta["Group"] = pd.factorize(meta["Group"])[0]
#initialize new dataset where arrays will go
meta_all = pd.DataFrame(columns = ["img_array","label","subject"])
create_dataset(meta, meta_all, path_to_datadir)

opened meta
442
ADNI_033_S_4508_MR_MT1__GradWarp__N3m_Br_20130314132259888_S182638_I363275.nii
flattened_ADNI2/ADNI_033_S_4508_MR_MT1__GradWarp__N3m_Br_20130314132259888_S182638_I363275.nii
ADNI_009_S_0842_MR_MT1__GradWarp__N3m_Br_20121204143339256_S172999_I349246.nii
flattened_ADNI2/ADNI_009_S_0842_MR_MT1__GradWarp__N3m_Br_20121204143339256_S172999_I349246.nii
ADNI_016_S_0702_MR_MT1__GradWarp__N3m_Br_20130618144352250_S189035_I377041.nii
flattened_ADNI2/ADNI_016_S_0702_MR_MT1__GradWarp__N3m_Br_20130618144352250_S189035_I377041.nii
ADNI_027_S_0307_MR_MT1__GradWarp__N3m_Br_20131017112700704_S193059_I394782.nii
flattened_ADNI2/ADNI_027_S_0307_MR_MT1__GradWarp__N3m_Br_20131017112700704_S193059_I394782.nii
ADNI_137_S_4351_MR_MT1__GradWarp__N3m_Br_20121213162425327_S177168_I351443.nii
flattened_ADNI2/ADNI_137_S_4351_MR_MT1__GradWarp__N3m_Br_20121213162425327_S177168_I351443.nii
ADNI_027_S_1045_MR_MT1__GradWarp__N3m_Br_20140623100854245_S213399_I432144.nii
flattened_ADNI2/ADNI_027_S_1045_MR_